In [1]:
!pip install GPy
!pip install pyGPs
!pip install rpy2

  Using cached paramz-0.9.5.tar.gz (71 kB)
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102556 sha256=2992c92e590b25e36b31390277dc34ae6dc6f88df0bd0e181a2ea44f311a2586
  Stored in directory: c:\users\dzerz\appdata\local\pip\cache\wheels\c8\95\f5\ce28482da28162e6028c4b3a32c41d147395825b3cd62bc810
Successfully built paramz
  Using cached pyGPs-1.3.5.tar.gz (10.3 MB)
  Created wheel for pyGPs: filename=pyGPs-1.3.5-py3-none-any.whl size=72636 sha256=ea39929fa21278ddbe99ba391239f120fdee4d926301c72557ea6863fbcef1ba
  Stored in directory: c:\users\dzerz\appdata\local\pip\cache\wheels\c1\2e\0e\1e78cf102e0a562c9a3c9ae5ab9bd28dbbb823235b7e0d82db
Successfully built pyGPs
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 

In [2]:
import scipy
import numpy as np
import pandas as pd
import sklearn
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.signal import convolve
from scipy.interpolate import interp1d
from scipy.sparse import csc_matrix
from typing import Dict, Any, List, Optional
from functools import reduce
from pyDOE import lhs
from multiprocessing import Pool, cpu_count
import multiprocessing
import GPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from rpy2.robjects.packages import importr
from rpy2.robjects import r, Formula
import inspect

from scipy.optimize import minimize
from multiprocessing import Pool, cpu_count

#import multiprocessing as mp
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler

from joblib import Parallel, delayed
import itertools

In [ ]:
# Gaussian mixture

def fn_gmm_mixtools(vals):
    # Add a small amount of noise to the data
    corrected_vals = vals + np.random.uniform(0, 0.0001, size=len(vals))

    try:
        # Fit a Gaussian mixture model with 3 components using the EM algorithm
        gmm = GaussianMixture(n_components=3, covariance_type='full', max_iter=300)
        gmm.fit(corrected_vals.reshape(-1, 1))

        return {
            'mu': gmm.means_.reshape(-1, 1),
            'sigma': np.sqrt(gmm.covariances_).reshape(-1, 1),
            'lambda': gmm.weights_.reshape(-1, 1),
        }
    except:
        return None

In [ ]:
# Gaussian mixture - finding peaks

def fn_gmm_mixtools_v2(vals):
    corrected_vals = vals + np.random.uniform(0, 0.0001, size=len(vals))
    try:
        kde = gaussian_kde(corrected_vals)
        x_vals = np.linspace(min(corrected_vals), max(corrected_vals), 1000)
        y_vals = kde(x_vals)
        df = np.column_stack((x_vals, y_vals))
        df = df[np.insert(np.diff(np.diff(df[:, 1])) >= 0, 0, False), :]    #finding peaks (where 2nd derivative changes sign)
        y_max = np.max(df[:, 1])
        y_max_limit = y_max * 0.2
        x_mean = df[df[:, 1] > y_max_limit, 0]
        
        gmm = GaussianMixture(n_components=3, means_init=np.reshape(x_mean, (-1, 1)), max_iter=300)
        gmm.fit(np.reshape(corrected_vals, (-1, 1)))
        
        return {"mu": np.reshape(gmm.means_, (-1, 1)),
                "sigma": np.reshape(np.sqrt(gmm.covariances_), (-1, 1)),
                "lambda": np.reshape(gmm.weights_, (-1, 1))}
    except:
        return None

In [ ]:
def fn_gmm_mclust(vals):
    corrected_vals = vals + np.random.uniform(0, 0.0001, len(vals))
    gmm_fit = GaussianMixture(n_components=1, covariance_type='full').fit(corrected_vals.reshape(-1, 1))

    mu_est = gmm_fit.means_
    sigma_est = np.sqrt(gmm_fit.covariances_)

    if len(mu_est) == 1:
        lambda_est = np.array([1])
    else:
        lambda_est = gmm_fit.weights_

    return {'mu': mu_est, 'sigma': sigma_est, 'lambda': lambda_est}

In [ ]:
def fn_gmm_kclust(vals):
    corrected_vals = vals + np.random.uniform(0, 0.0001, size=len(vals))
    num_clusters = None
    
    try:
        sil_scores = []
        for k in range(2, 6):                                  # not sure why 2-6 clusters
            kmeans = KMeans(n_clusters=k, init='k-means++')
            kmeans.fit(corrected_vals.reshape(-1, 1))
            sil_scores.append(silhouette_score(corrected_vals.reshape(-1, 1), kmeans.labels_))
        num_clusters = np.argmax(sil_scores) + 2
        
    except:
        num_clusters = 1
    
    if num_clusters == 1:
        clustered_stats = {
            'lambda': 1.0,
            'mu': np.mean(corrected_vals),
            'sigma': np.std(corrected_vals)
        }
    else:
        gmm_fit = GaussianMixture(n_components=num_clusters, covariance_type='full')
        gmm_fit.fit(corrected_vals.reshape(-1, 1))
        clustered_stats = []
        for i in range(num_clusters):
            lambda_est = gmm_fit.weights_[i]
            mu_est = gmm_fit.means_[i][0]
            sigma_est = np.sqrt(gmm_fit.covariances_[i][0])
            clustered_stats.append({'lambda': lambda_est, 'mu': mu_est, 'sigma': sigma_est})
    
    return {'mu': [x['mu'] for x in clustered_stats],
            'sigma': [x['sigma'] for x in clustered_stats],
            'lambda': [x['lambda'] for x in clustered_stats]}

In [ ]:
def fn_utility(utility_func, *args):
    return utility_func(*args)

In [ ]:
def fn_utility_ei(traits, optimal_response, complement_u=False, estimated_sigma=None):
    m = traits['mu']
    s = traits['sigma']
    
    ## If sigma is estimated with other other than empirical estimation
    if estimated_sigma is not None:
        s = estimated_sigma
    
    s[s[:, 0] == 0, 0] = np.nan
    
    gamma = (optimal_response - m) / s
    u_val_mat = s * (gamma * norm.cdf(gamma) + norm.pdf(gamma))
    u_val_mat = np.hstack((u_val_mat, np.zeros((u_val_mat.shape[0], 7))))
    
    if complement_u:
        list_gMix = fn_utility_global_gmix(traits, optimal_response)
        complement_utilities_fn = [
            {'f': fn_utility_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response}},
            {'f': fn_utility_gmix_agg_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_wmax_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_agg_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_wmax_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}}
        ]
        
        c_utilities_val = np.hstack([fn['f'](**fn['params']) for fn in complement_utilities_fn])
        u_val_mat = np.hstack((u_val_mat, c_utilities_val))
    
    return u_val_mat

In [ ]:
########################################################################################################################################################################################
########################################################################################################################################################################################

In [ ]:
############################################################################################
# Compute variance of estimate from a ranger model
# 
# Computes variances for a prediction from a ranger model, using the infinitesimal jackknife procedure
# 
# This function is a ranger-adaptation of the package \pkg{randomForestCI} of Wager et al. (2014). Their original can be found on github: \url{ https://github.com/swager/randomForestCI/}. 
#
# @param pred A nrow(newdata) by no. of trees matrix which contains numeric predictions
#        from a random forest trained with trees grown on bootstrap samples of the training data
# @param inbag A number of obs. in the training data by no. of trees matrix giving the
#        number of times the ith observation in the training data appeared in the bootstrap sample for the jth tree.
# @param calibrate whether to apply calibration to mitigate Monte Carlo noise
#        warning: if calibrate = FALSE, some variance estimates may be negative
#                 due to Monte Carlo effects if the number of trees in rf is too small
# @param used.trees set of trees to use for variance estimation; uses all tress if NULL
#
# @return A two-column matrix is returned, with predictions in the first column and estimates of prediction variance in the second. 
# @author Stefan Wager

In [ ]:
############ May needs repair .dot->@  #############

In [ ]:
def rInfJack(pred, inbag, calibrate=True, used_trees=None):
    """
    Compute the infinitesimal jackknife variance estimator for a random forest model.

    Args:
        pred (ndarray): An n x B matrix of tree-wise predictions for n observations and B trees.
        inbag (ndarray): An n x B matrix indicating which observations were included in each bootstrap sample.
        calibrate (bool): Whether to calibrate the variance estimates using empirical Bayes. Default is True.
        used_trees (list): A list of indices indicating which trees to use in the variance estimation. Default is all trees.

    Returns:
        ndarray: An n x 2 array of estimated mean and variance for each observation.
    """
    if used_trees is None:
        used_trees = range(pred.shape[1])

    pred = pred[:, used_trees]
    
    # Check if sampling without replacement
    no_replacement = (np.max(inbag) == 1)
    
    # Extract tree-wise predictions and variable counts from random forest
    B = len(used_trees)
    n = inbag.shape[0]
    s = np.sum(inbag, axis=1) / inbag.shape[1]
    
    y_hat = np.mean(pred, axis=1)
    pred_centered = pred - np.mean(pred, axis=1, keepdims=True)
    
    N = csr_matrix(inbag[:, used_trees])
    N_avg = N.mean(axis=1)
    
    # Compute raw infinitesimal jackknife
    if B ** 2 > n * pred.shape[0]:
        C = N.dot(pred_centered).T - N_avg.dot(np.sum(pred_centered, axis=0))
        raw_IJ = np.sum(C ** 2, axis=0) / B ** 2
    else:
    # Faster implementation when n is large. Uses the fact that colSums((A - B)^2) = T1 - 2 * T2 + T3,
    # where T1 = diag(A'A), T2 = diag(B'A), and T3 = diag(B'B)
        NTN = N.T.dot(N)
        NTNPT_T = pred_centered.T.dot(NTN)  # or NTNPT_T = pred_centered.T @ NTN
        T1 = np.sum(pred_centered * NTNPT_T, axis=0)
        
        RS = np.sum(pred_centered, axis=0)
        NbarTN = N_avg.T.dot(N)
        T2 = RS * NbarTN.dot(pred_centered)   #.T   or RS * NbarTN @ pred_centered
        
        T3 = np.sum(N_avg ** 2) * RS ** 2
        raw_IJ = (T1 - 2 * T2 + T3) / B ** 2
    
    # Apply Monte Carlo bias correction
    N_var = np.mean(N.power(2).mean(axis=1) - N_avg.power(2))
    boot_var = np.sum(pred_centered ** 2, axis=1) / B
    bias_correction = n * N_var * boot_var / B
    vars = raw_IJ - bias_correction
    
    # Finite sample correction
    if no_replacement:
        variance_inflation = 1 / (1 - np.mean(inbag, axis=1)) ** 2
        vars *= variance_inflation[:, np.newaxis]
    
    results = np.hstack((y_hat[:, np.newaxis], vars[:, np.newaxis]))
    
    if calibrate and results.shape[0] <= 20:
        calibrate = False
        print("Sample size <= 20, no calibration performed.")
    
                               

    if calibrate:
        # Compute variance estimates using half the trees
        calibration_ratio = 2
        n_sample = np.ceil(B / calibration_ratio).astype(int)
        #used_trees=np.random.choice(used.trees, n_sample, replace=False)
        results_ss = rInfJack(pred, inbag, calibrate=False, used_trees=np.random.choice(used_trees, n_sample, replace=False))

        # Use this second set of variance estimates to estimate scale of Monte Carlo noise
        sigma2_ss = np.mean((results_ss['var.hat'] - results['var.hat'])**2)
        delta = n_sample / B
        sigma2 = (delta**2 + (1 - delta)**2) / (2 * (1 - delta)**2) * sigma2_ss

        # Use Monte Carlo noise scale estimate for empirical Bayes calibration
        try:
            vars_calibrated = calibrateEB(vars, sigma2)
            results['var.hat'] = vars_calibrated
        except Exception as e:
            print(f"Calibration failed with error:\n{e}\nFalling back to non-calibrated variance estimates.")
            #warnings.warn(f"Calibration failed with error:\n{str(e)}\nFalling back to non-calibrated variance estimates.")
            results = rInfJack(pred, inbag, calibrate=False, used_trees=used_trees)

    return results



In [ ]:
# Fit an empirical Bayes prior in the hierarchical model
#     mu ~ G, X ~ N(mu, sigma^2)
#
# @param X a vector of observations
# @param sigma noise estimate
# @param p tuning parameter -- number of parameters used to fit G
# @param nbin tuning parameter -- number of bins used for discrete approximation
# @param unif.fraction tuning parameter -- fraction of G modeled as "slab"
#
# @return posterior density estimate g
#
# @section References:
# For more details about "g-estimation", see: B Efron. Two modeling strategies for
# empirical Bayes estimation. Stat. Sci., 29: 285-301, 2014.
# @author Stefan Wager

In [ ]:
##############################################
####  maybe @ should replaced by .dot() #####
############################################

def gfit(X, sigma, p=2, nbin=1000, unif_fraction=0.1):
    
    # Define the bin boundaries and width
    xvals = np.linspace(min(np.min(X) - 2 * np.std(X), 0), max(np.max(X) + 2 * np.std(X), np.std(X)), num=nbin)
    binw = xvals[1] - xvals[0]
    
    # Rotate the noise kernel so that it aligns with the zero point
    zero_idx = np.max(np.where(xvals <= 0))
    noise_kernel = np.exp(-0.5 * (xvals/sigma)**2) * binw / sigma
    noise_rotate = np.roll(noise_kernel, -zero_idx)
    
    # Generate the design matrix XX
    XX = np.column_stack([xvals**j * (xvals >= 0) for j in range(1, p+1)])
    
    # Define the negative log-likelihood function
    def neg_loglik(eta):
        g_eta_raw = np.exp(XX @ eta) * (xvals >= 0)
        if (np.sum(g_eta_raw) == np.inf) or (np.sum(g_eta_raw) <= 100 * np.finfo(np.double).eps):
            return 1000 * (len(X) + np.sum(eta**2))
        g_eta_main = g_eta_raw / np.sum(g_eta_raw)
        g_eta = (1 - unif_fraction) * g_eta_main + unif_fraction * (xvals >= 0) / np.sum(xvals >= 0)
        f_eta = convolve(g_eta, noise_rotate, mode='same')
        return np.sum(np.interp(X, xvals, -np.log(np.maximum(f_eta, 1e-7))))
    
    # Fit the model using the Nelder-Mead algorithm
    eta_hat = minimize(neg_loglik, np.repeat(-1, p), method='nelder-mead').x
    g_eta_raw = np.exp(XX @ eta_hat) * (xvals >= 0)
    g_eta_main = g_eta_raw / np.sum(g_eta_raw)
    g_eta = (1 - unif_fraction) * g_eta_main + unif_fraction * (xvals >= 0) / np.sum(xvals >= 0)
    
    # Return the estimated density function
    return {'x': xvals, 'g': g_eta}



In [ ]:
def gbayes(x0, gest, sigma):
    Kx = np.exp(-(gest['x'] - x0)**2 / (2 * sigma**2)) / np.sqrt(2 * np.pi * sigma**2)
    post = Kx * gest['g']
    post = post / np.sum(post)
    return np.sum(post * gest['x'])

In [ ]:
# Empirical Bayes calibration of noisy variance estimates
#
# @param vars list of variance estimates
# @param sigma2 estimate of the Monte Carlo noise in vars
#
# @return calibrated variance estimates
# @author Stefan Wager

In [16]:
def calibrateEB(vars, sigma2):
    if sigma2 <= 0 or np.min(vars) == np.max(vars):
        return np.maximum(vars, 0)
    
    sigma = np.sqrt(sigma2)
    eb_prior = gfit(vars, sigma)
    
    if len(vars) >= 200:
        # If there are many test points, use interpolation to speed up computations
        calib_x = np.unique(np.quantile(vars, q=np.arange(0, 1.02, 0.02)))
        calib_y = np.array([gbayes(xx, eb_prior, sigma) for xx in calib_x])
        f = interp1d(calib_x, calib_y, kind='cubic', fill_value='extrapolate')
        calib_all = f(vars)
    else:
        calib_all = np.array([gbayes(xx, eb_prior, sigma) for xx in vars])
    
    return calib_all

In [17]:
########################################################################################################################################################################################
########################################################################################################################################################################################

In [18]:
def fn_utility_ei_ij(traits, optimal_response, complement_u=False):
    m = traits['mu']
    s = traits['sigma']
    
    # Compute IJ estimator for variance
    inbag = np.column_stack(traits['custom']['inbag'])
    sigma_IJ = rInfJack(traits['values'], inbag, calibrate=True)['var.hat']
    
    # If sigma is estimated with other than empirical estimation
    if sigma_IJ is not None:
        s = np.array(sigma_IJ)
    
    s[s == 0] = np.nan
    
    gamma = (optimal_response - m) / s
    u_val_mat = s * (gamma * norm.cdf(gamma) + norm.pdf(gamma))
    u_val_mat = np.column_stack((u_val_mat, np.zeros(len(m))))  # add column for complement utility
    
    if complement_u:
        list_gMix = fn_utility_global_gmix(traits, optimal_response)
        
        complement_utilities_fn = [
            {'f': fn_utility_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response}},
            {'f': fn_utility_gmix_agg_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_wmax_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_agg_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_wmax_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}}
        ]
        
        c_utilities_val = np.column_stack([u_f['f'](**u_f['params']) for u_f in complement_utilities_fn])
        u_val_mat = np.column_stack((u_val_mat, c_utilities_val))
    
    return u_val_mat

In [19]:
def fn_utility_lcb(traits: Dict[str, np.ndarray], optimal_response: np.ndarray, kappa: float = 2.0, complement_u: bool = False, estimated_sigma: Optional[np.ndarray] = None) -> np.ndarray:
    m = traits['mu']
    s = traits['sigma']
    
    ## If sigma is estimated with other than empirical estimation
    if estimated_sigma is not None:
        s = estimated_sigma
    
    u_val = -1 * (m - kappa * s)
    u_val_mat = np.tile(u_val, (len(m), 1)).T
    u_val_mat = u_val_mat.astype(float)
    u_val_mat = np.hstack([u_val_mat])
    
    if complement_u:
        list_gMix = fn_utility_global_gmix(traits, optimal_response)
        
        complement_utilities_fn = [
            {"f": fn_utility_ei, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_gmix_agg_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_wmax_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_agg_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_wmax_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}}
        ]
        
        c_utilities_val = np.hstack([u_f["f"](**u_f["params"]) for u_f in complement_utilities_fn])
        u_val_mat = np.hstack([u_val_mat, c_utilities_val])
    
    return u_val_mat

In [20]:
def fn_utility_lcb_ij(traits, optimal_response, complement_u = False):
    sigma_IJ <- rInfJack(traits['values'], np.column_stack(traits['custom']['inbag']), calibrate = True)
    return fn_utility_lcb(traits, optimal_response, complement_u, estimated_sigma = float(sigma_IJ['var.hat']))

In [21]:
def fn_utility_global_gmix(traits, optimal_response, byrow=True):
    apply_margin = 1 if byrow else 0
    def gmix_fn(vals):
        gmix = GaussianMixture(n_components=1).fit(vals.reshape(-1, 1))
        mu, sigma = gmix.means_, np.sqrt(gmix.covariances_)
        lambda_ = np.array([1.])
        return {'mu': mu, 'sigma': sigma, 'lambda': lambda_}
    return np.apply_along_axis(gmix_fn, axis=apply_margin, arr=traits['values'])

In [22]:
def fn_utility_gmix_agg_ei(traits, optimal_response, byrow=True, complement_u=False, list_MM=None):
    if list_MM is None:
        list_gMix = fn_utility_global_gmix(traits, optimal_response, byrow)
    else:
        list_gMix = list_MM
        
    # Integral part of the utility calculation
    utilities_gMix = np.vstack([
        np.dot(single_gMix.rx2("lambda")[:, 0], fn_utility_ei(
            traits,
            optimal_response,
            estimated_sigma=np.asarray(single_gMix.rx2("var.hat"))
        ))
        for single_gMix in list_gMix
    ]).T
    utilities_gMix = pd.DataFrame(utilities_gMix, columns=["gmix_agg_ei"])
    
    if complement_u:
        complement_utilities_fn = [
            {"f": fn_utility_ei, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_lcb, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_gmix_wmax_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_agg_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_wmax_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}}
        ]
        
        c_utilities_val = pd.concat([
            pd.DataFrame(do.call(u_f["f"], u_f["params"])).T
            for u_f in complement_utilities_fn
        ], axis=1)
        utilities_gMix = pd.concat([utilities_gMix, c_utilities_val], axis=1)
        
    return utilities_gMix

In [23]:
def fn_utility_gmix_wmax_ei(traits, optimal_response, byrow=True, complement_u=False, list_MM=None):
    list_gMix = list_MM
    if list_gMix is None:
        list_gMix = fn_utility_global_gmix(traits, optimal_response, byrow)
    
    # Integral part of the utility calculation
    utilities_gMix = np.vstack([single_gMix['lambda'][:, 0] * fn_utility_ei(single_gMix, optimal_response)[:, 0]
                                for single_gMix in list_gMix])
    util_gMix_max_idx = np.argmax(utilities_gMix, axis=0)
    utilities_gMix = np.multiply(utilities_gMix[util_gMix_max_idx, :], list_gMix[util_gMix_max_idx]['lambda'][:, 0])
    utilities_gMix = utilities_gMix.reshape(-1, 1)
    utilities_gMix = np.column_stack((utilities_gMix,))

    colnames = ["gmix_wmax_ei"]
    
    if complement_u:
        complement_utilities_fn = [
            {"f": fn_utility_ei, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_lcb, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_gmix_agg_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_agg_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_wmax_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}}
        ]
        
        c_utilities_val = np.hstack([do_call(u_f["f"], **u_f["params"]) for u_f in complement_utilities_fn])
        utilities_gMix = np.column_stack((utilities_gMix, c_utilities_val))

    return pd.DataFrame(utilities_gMix, columns=colnames)

In [24]:
def fn_utility_gmix_max_ei(traits, optimal_response, byrow=True, complement_u=False, list_MM=None):
    list_gMix = list_MM
    if list_gMix is None:
        list_gMix = fn_utility_global_gmix(traits, optimal_response, byrow)
    
    utilities_gMix = np.concatenate([fn_utility_ei(single_gMix, optimal_response)[np.argmax(fn_utility_ei(single_gMix, optimal_response)), None] 
                                     for single_gMix in list_gMix], axis=0)
    utilities_gMix = utilities_gMix.reshape(-1, 1)
    utilities_gMix = np.hstack([utilities_gMix])
    utilities_gMix = np.hstack([utilities_gMix, np.zeros((traits.shape[0], 0))])
    
    if complement_u:
        complement_utilities_fn = [
            {'f': fn_utility_ei, 'params': {'traits': traits, 'optimal_response': optimal_response}},
            {'f': fn_utility_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response}},
            {'f': fn_utility_gmix_agg_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_agg_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_wmax_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}}
        ]
        
        c_utilities_val = np.hstack([f['f'](**f['params']) for f in complement_utilities_fn])
        utilities_gMix = np.hstack([utilities_gMix, c_utilities_val])
    
    return utilities_gMix

In [25]:
def fn_utility_gmix_agg_lcb(traits, optimal_response, byrow=True, complement_u=False, list_MM=None):
    if list_MM is None:
        list_gMix = fn_utility_global_gmix(traits, optimal_response, byrow)
    else:
        list_gMix = list_MM

    # Integral part of the utility calculation
    utilities_gMix = np.vstack([np.dot(single_gMix['lambda'].T, fn_utility_lcb(single_gMix, optimal_response)) for single_gMix in list_gMix])
    utilities_gMix = np.column_stack((utilities_gMix, np.array([["gmix_agg_lcb"]*utilities_gMix.shape[0]]).T))

    if complement_u:
        complement_utilities_fn = [
            {"f": fn_utility_ei, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_lcb, "params": {"traits": traits, "optimal_response": optimal_response}},
            {"f": fn_utility_gmix_agg_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_wmax_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_ei, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_wmax_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}},
            {"f": fn_utility_gmix_max_lcb, "params": {"traits": traits, "optimal_response": optimal_response, "list_MM": list_gMix}}
        ]
        c_utilities_val = reduce(lambda x, y: np.column_stack((x, y)), [f["f"](**f["params"]) for f in complement_utilities_fn])
        utilities_gMix = np.column_stack((utilities_gMix, c_utilities_val))

    return utilities_gMix

In [26]:
def fn_utility_gmix_wmax_lcb(traits, optimal_response, byrow=True, complement_u=False, list_MM=None):
    list_gMix = list_MM
    if list_gMix is None:
        list_gMix = fn_utility_global_gmix(traits, optimal_response, byrow)

    # Integral part of the utility calculation
    utilities_gMix = np.concatenate([base_utility(single_gMix, optimal_response)[:, np.argmax(base_utility(single_gMix, optimal_response)[:, 0])] * single_gMix['lambda'][np.argmax(base_utility(single_gMix, optimal_response)[:, 0])] for single_gMix in list_gMix]).reshape(-1, 1)
    utilities_gMix = np.hstack((utilities_gMix,))

    if complement_u:
        complement_utilities_fn = [
            {'f': fn_utility_ei, 'params': {'traits': traits, 'optimal_response': optimal_response}},
            {'f': fn_utility_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response}},
            {'f': fn_utility_gmix_agg_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_wmax_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_ei, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_agg_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}},
            {'f': fn_utility_gmix_max_lcb, 'params': {'traits': traits, 'optimal_response': optimal_response, 'list_MM': list_gMix}}
        ]

        c_utilities_val = np.column_stack([fn['f'](**fn['params']) for fn in complement_utilities_fn])
        utilities_gMix = np.column_stack((utilities_gMix, c_utilities_val))

    return utilities_gMix

In [27]:
def fn_utility_gmix_max_lcb(traits, optimal_response, byrow=True, complement_u=False, list_MM=None):
    if list_MM is None:
        list_MM = fn_utility_global_gmix(traits, optimal_response, byrow)

    ## Integral part of the utility calculation
    utilities_gMix = np.vstack([single_gMix['lambda'] * base_utility(single_gMix, optimal_response)[util_gMix.argmax(axis=0)[0], :] for single_gMix in list_gMix])
    utilities_gMix = utilities_gMix[:, np.newaxis] # add a singleton dimension to match R's output format
    utilities_gMix = np.column_stack((utilities_gMix, ))
    utilities_gMix.columns = ['gmix_wmax_lcb']

    if complement_u:
        complement_utilities_fn = [
            dict(f=fn_utility_ei, params=dict(traits=traits, optimal_response=optimal_response)),
            dict(f=fn_utility_lcb, params=dict(traits=traits, optimal_response=optimal_response)),
            dict(f=fn_utility_gmix_agg_ei, params=dict(traits=traits, optimal_response=optimal_response, list_MM=list_gMix)),
            dict(f=fn_utility_gmix_wmax_ei, params=dict(traits=traits, optimal_response=optimal_response, list_MM=list_gMix)),
            dict(f=fn_utility_gmix_max_ei, params=dict(traits=traits, optimal_response=optimal_response, list_MM=list_gMix)),
            dict(f=fn_utility_gmix_agg_lcb, params=dict(traits=traits, optimal_response=optimal_response, list_MM=list_gMix)),
            dict(f=fn_utility_gmix_wmax_lcb, params=dict(traits=traits, optimal_response=optimal_response, list_MM=list_gMix)),
        ]

        c_utilities_val = np.column_stack([fn(**params) for fn, params in complement_utilities_fn])
        utilities_gMix = np.column_stack((utilities_gMix, c_utilities_val))
        utilities_gMix.columns = ['gmix_wmax_lcb', 'ei', 'lcb', 'gmix_agg_ei', 'gmix_wmax_ei', 'gmix_max_ei', 'gmix_agg_lcb', 'gmix_max_lcb']

    return utilities_gMix

In [28]:
def fn_eval_utility(ds, surrogate_predict_fn, surrogate_model, surrogate_utility_fn, optimal_value, col_names, direction=1, is_normalization_required=False, param_def=None, eval_complement=False):
    if ds.ndim == 1:
        ds = np.reshape(ds, (1, -1))
    ds = pd.DataFrame(ds, columns=col_names)

    if is_normalization_required:
        ds = fn_normalize_min_max(ds, param_def)[0]

    surrogate_predictions = fn_predict(surrogate_predict_fn, surrogate_model, ds, param_def=param_def)
    surrogate_traits = fn_assemble_prediction_traits(surrogate_predictions)
    s_utility = surrogate_utility_fn(traits=surrogate_traits, optimal_response=optimal_value, complement_u=eval_complement)
    
    s_utility = np.array(s_utility, dtype=float) * direction

    return s_utility

In [29]:
## Function that generates new points in space (parameter vector)
# Params:
# - type: function name to be called
# - params: tible with parameters
# - omit: vector of params to be omitted and use current_value
# - variation.rate: percentage of attributes to be changed

def fn_acquisition(type, param_space, *args, **kwargs):
    ## If space has no records, it is initial parameter generation, hence need completely random sampling
    # if nrow(param_space$space) == 0:
    #     return fn_acquisition_random_share(param_space, ...)
    
    return type(param_space, *args, **kwargs)



In [30]:
def fn_sampling_lhs(vals=None, param_space=None, sample_size=None):
    dim_names = param_space['definition']['parameter']
    sample_dim = param_space['definition'].shape[0]
    
    prior_sample = lhs(sample_dim, samples=sample_size)  # or improvedLHS()
    prior_sample = np.asarray(prior_sample)
    prior_sample = prior_sample * (param_space['definition']['max'].values - param_space['definition']['min'].values) + param_space['definition']['min'].values
    prior_sample = dict(zip(dim_names, prior_sample.T))
    
    return prior_sample

In [31]:

def fn_sampling_normal_optimal(vals, param_space, sample_size, **kwargs):
    dim_names = param_space['definition']['parameter']
    sample_dim = param_space['definition'].shape[0]
    sampling_intensity = kwargs.get('MUTATION_RATE', 0.5)
    sampling_deviation = kwargs.get('MUTATION_SD', 1)
  
    if 'OPTIMISATION_CORES' in kwargs and kwargs['OPTIMISATION_CORES'] > 1:
        with Pool(kwargs['OPTIMISATION_CORES']) as p:
            genereted_sample = np.hstack(p.starmap(generate_sample, [(i, param_space['definition'], sample_size, sampling_intensity, sampling_deviation, vals) for i in range(sample_dim)]))
    else:
        genereted_sample = np.hstack([generate_sample(i, param_space['definition'], sample_size, sampling_intensity, sampling_deviation, vals) for i in range(sample_dim)])
  
    genereted_sample = genereted_sample.transpose()
    genereted_sample = dict(zip(dim_names, genereted_sample))
    
    return genereted_sample

def generate_sample(c_i, p_defs, s_size, s_intensity, s_sd, o_values):
    p_def = p_defs.iloc[c_i]
    p_lower_bound = p_def['lower_limit']
    p_upper_bound = p_def['upper_limit']
    p_replaced_size = int(np.ceil(s_size * s_intensity))
    p_current = o_values[np.random.randint(o_values.shape[0]), c_i] # In case currently most optimal sets contains more than one sample set, then sample randomly one of them.
    p_sample = np.full(s_size, p_current)
    p_sample_replace = np.random.choice(s_size, p_replaced_size, replace=False)
  
    #p_sample[p_sample_replace] = np.random.uniform(p_lower_bound, p_upper_bound, p_replaced_size) # UNIFORMLY - SAMPLING
    p_sample[p_sample_replace] = np.random.normal(p_current, s_sd, p_replaced_size) # NORMAL AROUND THE OPTIMAL - SAMPLING
      
    return p_sample

In [32]:
def fn_optimisation_uniform_prior(vals, sampling_fn, surrogate_predict_fn, surrogate_model, surrogate_utility_fn, iteration_space, param_range, param_space, omit, target, sample_size=1, variation_rate=0.3, opt_parallel=True, opt_cores=cpu_count()-2, opt_store_intermediate=True, is_normalization_required=False, eval_complement=False, **kwargs):
    
    prior_sample = sampling_fn(vals, param_space, sample_size, **kwargs)
    prior_names = list(prior_sample.columns)
    
    if opt_parallel:
        sample_chunks = np.array_split(prior_sample, opt_cores)
        
        with Pool(opt_cores) as pool:
            eval_targets = pool.map(lambda s_chunk: fn_eval_utility(
                s_chunk,
                surrogate_predict_fn=surrogate_predict_fn,
                surrogate_model=surrogate_model,
                surrogate_utility_fn=surrogate_utility_fn,
                optimal_value=iteration_space['optimal']['value'],
                col_names=list(vals.keys()),
                is_normalization_required=is_normalization_required,
                param_def=param_space['definition'],
                eval_complement=eval_complement
            ), sample_chunks)
        
        eval_target = pd.concat(eval_targets, axis=0)
    else:
        eval_target = fn_eval_utility(
            prior_sample,
            surrogate_predict_fn=surrogate_predict_fn,
            surrogate_model=surrogate_model,
            surrogate_utility_fn=surrogate_utility_fn,
            optimal_value=iteration_space['optimal']['value'],
            col_names=list(vals.keys()),
            is_normalization_required=is_normalization_required,
            param_def=param_space['definition'],
            eval_complement=eval_complement
        )
    
    prior_sample[target] = eval_target.iloc[:, 0].values
    
    if eval_complement and eval_target.shape[1] > 1:
        prior_sample[eval_target.columns] = eval_target.values
    
    best_val_idx = prior_sample[target].idxmax()
    optimal_set = prior_sample.drop(target, axis=1).iloc[best_val_idx]
    
    output_obj = {}
    output_obj['space'] = optimal_set.to_frame().T
    output_obj['space'].columns = list(vals.keys())
    output_obj['utility'] = prior_sample[target].iloc[best_val_idx]
    
    if opt_store_intermediate or eval_complement:
        output_obj['pop'] = prior_sample
        output_obj['pop_utility'] = prior_sample[target]
        
        if eval_complement:
            output_obj['complementary_utility_fn'] = eval_target.columns.drop([target] + prior_names)
    
    return output_obj

In [33]:
def fn_optimisation_quasi_newthon(
    vals, 
    sampling_fn, 
    surrogate_predict_fn, 
    surrogate_model, 
    surrogate_utility_fn, 
    iteration_space, 
    param_range, 
    param_space, 
    sample_size=1, 
    opt_parallel=True, 
    opt_cores=cpu_count()-2, 
    opt_best_init=False, 
    opt_init_pop_shufle_rate=0.1, 
    opt_store_intermediate=False, 
    is_normalization_required=False,
    **kwargs
):
    enabled_idx = np.arange(len(param_space["definition"]))
    if "enabled" in param_space["definition"]:
        enabled_idx = np.where(param_space["definition"]["enabled"] == True)[0]
        
    reduced_vals = vals[enabled_idx]
    reduced_param_space = {
        "space": param_space["space"][:, np.concatenate([enabled_idx, [-1]])],
        "definition": param_space["definition"][enabled_idx]
    }
    
    lower_bound = reduced_param_space["definition"]["lower_limit"]
    upper_bound = reduced_param_space["definition"]["upper_limit"]
    
    def singleton_optimisation(idx, init_grid):
        optimal_set = minimize(
            fun=fn_eval_utility, 
            x0=init_grid[idx],
            args=(surrogate_predict_fn, surrogate_model, surrogate_utility_fn, iteration_space["optimal"]["value"], list(reduced_vals), is_normalization_required, reduced_param_space["definition"]),
            method="L-BFGS-B",
            bounds=list(zip(lower_bound, upper_bound)),
            options={"maxiter": 1000},
        )
        return optimal_set
    
    initGrid = sampling_fn(reduced_vals, reduced_param_space, sample_size, **kwargs)
    
    if opt_parallel:
        with Pool(processes=opt_cores) as pool:
            optimal_sets = pool.starmap(singleton_optimisation, [(idx, initGrid) for idx in range(sample_size)])
    else:
        optimal_sets = [singleton_optimisation(idx, initGrid) for idx in range(sample_size)]
    
    optimal_sets = [opt_set for opt_set in optimal_sets if opt_set.success]
    best_vals = np.array([opt_set.fun for opt_set in optimal_sets])
    best_val_idx = np.where(best_vals == best_vals.max())[0]
    
    if len(best_val_idx) > 1:
        best_val_idx = best_val_idx[0]
    
    optimal_set = optimal_sets[best_val_idx]
    
    output_obj = {
        "space": np.concatenate([param_space["definition"]["initial"][enabled_idx], optimal_set.x]),
        "utility": optimal_set.fun
    }
    
    output_obj["space"] = output_obj["space"].reshape((1, -1))
    output_obj["space"] = np.tile(output_obj["space"], (sample_size, 1))
    output_obj["space"][:, enabled_idx] = np.tile(optimal_set.x, (sample_size, 1))
    
    if opt_store_intermediate:
        output_obj["pop"] = np.array([opt_set.x for opt_set in optimal_sets])
        output_obj["pop_utility"] = best_vals
    
    return output_obj

In [34]:
from scipy.optimize import differential_evolution

In [35]:
def fn_optimisation_differential_evolution(vals, surrogate_predict_fn, surrogate_model, surrogate_utility_fn, iteration_space, param_range, param_space, sample_size = 1, opt_parallel = True, opt_cores = multiprocessing.cpu_count()-2, opt_best_init = False, opt_init_pop_shufle_rate = 0.1, opt_store_intermediate = False, is_normalization_required = False, **kwargs):
    
    enabled_idx = np.arange(param_space['definition'].shape[0])
    if 'enabled' in param_space['definition']:
        enabled_idx = np.where(param_space['definition'][:, 'enabled'] == True)[0]
    
    reduced_vals = vals[enabled_idx]
    reduced_param_space = {
        'space': param_space['space'][:, np.concatenate([enabled_idx, [-1]])],
        'definition': param_space['definition'][enabled_idx]
    }
    
    lower_bound = reduced_param_space['definition']['lower_limit']
    upper_bound = reduced_param_space['definition']['upper_limit']
    
    arg_params = {
        'strategy': 6, 
        'p': 0.01, 
        'F': 0.8, 
        'c': 0.8, 
        'itermax': 1000, 
        'NP': 10*len(enabled_idx), 
        'trace': False
    }
    
    if opt_store_intermediate:
        arg_params['storepopfrom'] = 1
        arg_params['storepopfreq'] = 10
    
    optimal_sets = []
    for x in range(sample_size):
        if opt_parallel:
            cl = multiprocessing.Pool(processes=opt_cores)
            arg_params['parallelType'] = 1
            arg_params['cluster'] = cl
            cl.map(lambda fn: __import__(fn), ['numpy', 'pandas', 'random', 'scipy'])
            cl.map(lambda pkg: __import__(pkg), ['tidyr', 'dplyr', 'ggplot2', 'reshape2', 'purrr', 'GPfit', 'randomForest', 'mixtools', 'mclust', 'NbClust', 'mlegp', 'ranger'])
        
        opt_output = differential_evolution(
            fn_eval_utility, 
            bounds=list(zip(lower_bound, upper_bound)),
            strategy=arg_params['strategy'], 
            popsize=arg_params['NP'], 
            tol=arg_params['p'], 
            mutation=arg_params['F'], 
            recombination=arg_params['c'], 
            maxiter=arg_params['itermax'], 
            args=(
                surrogate_predict_fn, 
                surrogate_model, 
                surrogate_utility_fn,
                iteration_space['optimal']['value'],
                reduced_vals,
                is_normalization_required,
                reduced_param_space['definition'],
                -1
            )
        )
        if opt_parallel:
            cl.close()
        
        optimal_sets.append(opt_output)
    
    best_vals = np.array([opt['fun'] for opt in optimal_sets])
    best_val_idx = np.where(best_vals == np.max(best_vals))[0]
    
    if len(best_val_idx) > 1:
        best_val_idx = best_val_idx[0]
    optimal_set = optimal_sets[best_val_idx]

    output_obj = {}
    output_obj['space'] = list(param_space['definition'][:, 'initial'])
    output_obj['space'][enabled_idx] = optimal_set['optim']['bestmem']
    output_obj['space'] = np.matrix(output_obj['space'], nrow=1)
    output_obj['space'].columns = list(vals.keys())

    if opt_store_intermediate:
        output_obj['pop'] = optimal_set['member']['storepop']

    return output_obj

In [36]:
def fn_reference_sample(sps, ref_type=0, target=None, **kwargs):
    assert ref_type in [0, 1], "Chosen type of a reference sample point is not correct. Allowed values are either 0 ('optimal') or 1 ('last')."

    if sps.shape[0] > 0:
        if ref_type == 0:
            return sps.loc[sps[target].idxmin()]
        elif ref_type == 1:
            return sps.tail(1)

    return None

# assertthat package is not imported

In [37]:
def fn_optimal_acquisition(param_space, iter_space = None, sample_size = 1, omit = [], inner_optimisation_fn = fn_optimisation_uniform_prior, sampling_fn = fn_sampling_lhs, surrogate_predict_fn = None, surrogate_utility_fn = None, **kwargs):
    assert callable(inner_optimisation_fn), "Provided optimisation function is not a valid function!"
    assert callable(sampling_fn), "Provided sampling function is not a valid function!"

    sps = param_space['space']
    # Bounds (min and max) are taken by default from the first parameter
    if param_space['definition'].shape[0] > 0:
        lower_limit = float(param_space['definition'].loc["lower_limit"])
        upper_limit = float(param_space['definition'].loc["upper_limit"])
    else:
        lower_limit = -10
        upper_limit = 10
    
    # Get reference sample point for future mutation
    ref_param_instance = fn_reference_sample(sps, ref_type = 0, **kwargs)

    # Omit any not necessary parameters
    if omit:
        ref_param_instance = ref_param_instance.drop(columns=omit)

    optimal_ps = inner_optimisation_fn(ref_param_instance, 
                                       sampling_fn = sampling_fn,
                                       surrogate_predict_fn = surrogate_predict_fn, 
                                       surrogate_model = iter_space['surrogate']['model'], 
                                       surrogate_utility_fn = surrogate_utility_fn, 
                                       iteration_space = iter_space, 
                                       param_range = {"lower_limit": lower_limit, "upper_limit": upper_limit},
                                       param_space = param_space,
                                       sample_size = sample_size,
                                       omit = omit,
                                       **kwargs)
    return optimal_ps

In [38]:
def fn_enforce_bounds(value, param, param_def):
    pps = param_def[param_def['parameter'] == param]
    
    if len(pps) == 0:
        return value
    if pps.iloc[0]['lower_limit'] > value:
        return pps.iloc[0]['lower_limit']
    if pps.iloc[0]['upper_limit'] < value:
        return pps.iloc[0]['upper_limit']
    
    return value

In [39]:

def fn_normalize_min_max(ds, param_def):
    if ds.ndim == 1:
        ds = pd.DataFrame(ds).T
        
    param_mat = param_def[['parameter', 'lower_limit', 'upper_limit']].set_index('parameter')
    params_used = list(set(param_mat.index) & set(ds.columns))
    params_omitted = list(set(ds.columns) - set(params_used))
    
    input_space_cols = []
    for col_name in params_used:
        col = ds[col_name]
        col_min = param_mat.at[col_name, 'lower_limit']
        col_max = param_mat.at[col_name, 'upper_limit']
        input_space_cols.append((col - col_min) / (col_max - col_min))
    
    input_space = pd.concat(input_space_cols, axis=1)
    input_space.columns = params_used
    
    if len(params_omitted) > 0:
        ds_omitted = ds[params_omitted]
        input_space = pd.concat([input_space, ds_omitted], axis=1)
        
    return input_space

In [40]:
def fn_scaleup_standard(ds, param_def):
    if ds.ndim == 1:
        ds = pd.DataFrame(ds).T
        ds.columns = param_def['parameter']
    param_mat = param_def[['parameter', 'lower_limit', 'upper_limit']].set_index('parameter')
    params_used = list(set(ds.columns).intersection(set(param_mat.index)))
    params_omitted = list(set(ds.columns).difference(set(params_used)))
    
    input_space_cols = []
    for col_name in params_used:
        l_lower = param_mat.loc[col_name, 'lower_limit']
        l_upper = param_mat.loc[col_name, 'upper_limit']
        l_range = l_upper - l_lower
        input_space_cols.append(((ds[col_name] * l_range) + l_lower).values)
        
    i_space = np.column_stack(input_space_cols)
    colnames = params_used
    if len(params_omitted) > 0:
        i_space = np.column_stack((i_space, ds[params_omitted].values))
        colnames += params_omitted
    i_space = pd.DataFrame(i_space, columns=colnames)
    
    return i_space

In [41]:
## Function to fit GP model
# Params:
# - ds: dataset (tibble)
# - target: target variable name

def fn_fit_gp(ds, target, method_params = {'KERNEL': 'exponential', 'POWER': 1.95}, test_ds = None):
    design_mat = ds.drop(target, axis=1).values
    
    model = GPy.models.GPRegression(
        X=design_mat,
        Y=ds[target].values.reshape(-1, 1),
        kernel=GPy.kern.RBF(input_dim=design_mat.shape[1], ARD=True, lengthscale=method_params['POWER'])
    )
    
    model.optimize()
    
    output = {'model': model}
    
    if test_ds is not None:
        test_ds_output = test_ds[target].values
        y_pred, _ = model.predict(test_ds.drop(target, axis=1).values)
        output['performance'] = np.sqrt(np.mean(np.square(y_pred - test_ds_output)))
    
    return output

In [42]:

# from pyGPs import mlegp

# def fn_fit_mlegp(ds, target, method_params={}, test_ds=None):
#     output = {
#         'model': mlegp(X=ds.drop(columns=[target]).values, Y=ds[target].values, **method_params)
#     }
#     if test_ds is not None:
#         test_ds_output = test_ds[target].values
#         y_pred = output['model'].predict(test_ds.drop(columns=[target]).values)[0]
#         output['performance'] = RMSE(y_pred=y_pred, y_true=test_ds_output)  
#     return output

# from pyGPs import mlegp
# from sklearn.metrics import mean_squared_error

# def fn_fit_mlegp(ds, target, method_params={}, test_ds=None):
#     output = {'model': mlegp(X=ds.drop(columns=target).values, Y=ds[target].values, **method_params)}    
#     if test_ds is not None:
#         y_true = test_ds[target].values
#         y_pred = output['model'].predict(test_ds.drop(columns=target).values)[0]
#         output['performance'] = mean_squared_error(y_true, y_pred, squared=False)   
#     return output

In [43]:
## Function to fit GP model (with MLEGP library)
# Params:
# - ds: dataset (tibble)          R
# - target: target variable name
# •	a dataset ds             Python
# •	the name of the target variable target
# •	optional parameters for the Gaussian process model method_params


def fn_fit_mlegp(ds, target, method_params = {}, test_ds = None):
    X = ds.loc[:, ds.columns != target].values
    y = ds[target].values.reshape(-1,1)
    kernel = GPy.kern.RBF(input_dim=X.shape[1], ARD=True)  # Radial basis function kernel with Automatic Relevance Determination
    model = GPy.models.GPRegression(X, y, kernel)
    model.optimize()
    output = {'model': model}
    
    if test_ds is not None:
        test_X = test_ds.loc[:, test_ds.columns != target].values
        test_y = test_ds[target].values.reshape(-1,1)
        y_pred = model.predict(test_X)[0]
        output['performance'] = np.sqrt(((y_pred - test_y) ** 2).mean())  ### May need to delet np.sqrt()
    
    return output

In [44]:
## Function to fit Random Forest model
# Params:
# ds: a Pandas DataFrame containing the training data.      
# target: a string representing the name of the target variable in the training data.
# method_params: a dictionary containing hyperparameters for the Random Forest model. The default values are RF_NSIZE=1 (minimum number of samples required to split an internal node), RF_NTREE=500 (number of trees in the forest), and RF_MTRY (number of features to consider when looking for the best split at each node), which is set to the square root of the number of features by default.
# test_ds: a Pandas DataFrame containing the test data. This argument is optional.
# keep_inbag: a boolean indicating whether to compute and store the "in-bag" score (i.e., the prediction error on the training set).

def fn_fit_rf(ds, target, method_params={"RF_NSIZE": 1, "RF_NTREE": 500}, test_ds=None, keep_inbag=True):
    
    if method_params.get("RF_MTRY") is None:
        method_params["RF_MTRY"] = int(np.sqrt(ds.shape[1] - 1))
    
    X = ds.drop(target, axis=1).values
    y = ds[target].values
    
    model = RandomForestRegressor(n_estimators=method_params["RF_NTREE"], 
                                   min_samples_leaf=method_params["RF_NSIZE"], 
                                   max_features=method_params["RF_MTRY"], 
                                   oob_score=keep_inbag)
    model.fit(X, y)
    
    output = {"model": model}
    
    if test_ds is not None:
        y_true = test_ds[target].values
        X_test = test_ds.drop(target, axis=1).values
        y_pred = model.predict(X_test)
        output["performance"] = np.sqrt(mean_squared_error(y_true, y_pred))
    
    return output

In [45]:
## Function to fit Random Forest model (ranger package)
# Params:
# dataset (ds) 
# target variable (target)
# optional arguments for method_params
# test_ds 
# keep_inbag

def fn_fit_rf_ranger(ds, target, method_params={"RF_NSIZE": 1, "RF_NTREE": 500}, test_ds=None, keep_inbag=True):
    
    if method_params.get("RF_MTRY") is None:
        method_params["RF_MTRY"] = int(np.sqrt(ds.shape[1] - 1))
    
    ranger = importr("ranger")
    
    formula = Formula(f"{target} ~ .")
    model = ranger.ranger(formula=formula, data=ds, num_trees=method_params["RF_NTREE"],
                          mtry=method_params["RF_MTRY"], min_node_size=method_params["RF_NSIZE"],
                          keep_inbag=keep_inbag)
    
    output = {"model": model}
    
    if test_ds is not None:
        y_true = test_ds[target].values
        X_test = test_ds.drop(target, axis=1).values
        y_pred = ranger.predict(model, X_test)
        output["performance"] = np.sqrt(mean_squared_error(y_true, y_pred))
    
    return output


In [46]:
## Function for creating folds for cross validation
## Params:
# ds: the dataset to perform cross-validation on
# folds: the number of folds to use (default is 10)
# loo: whether to use leave-one-out cross-validation (default is False)


def fn_do_folding(ds, folds=10, loo=False):
    if nrow(ds) <= folds:
        folds = nrow(ds)
        loo = True
    
    test_ds_idx = []
    
    if loo:
        test_ds_idx = [list(x) for x in np.random.choice(range(nrow(ds)), size=folds, replace=False)]
    else:
        fold_size = nrow(ds) // folds
        test_ds_idx = [list(x) for x in np.random.choice(range(nrow(ds)), size=fold_size, replace=True) for i in range(folds)]
    
    return [ {
        'train': ds.iloc[np.setdiff1d(range(nrow(ds)), idx), :],
        'test': ds.iloc[idx, :]
    } for idx in test_ds_idx ]

## numpy's random.choice function is used to generate the indices of the test sets. 
#  ds must be a pandas dataframe.

In [47]:
## Function to fit a model
# Params:
# - ds: dataset (tibble)
# - target: target variable name


# if it doesn t work -> import multiprocessing as mp. I change it cause of imports ... its used elsewhere with the name multiprocessing

def fn_fit(type, ds, target, param_def, is_normalization_required=False, method_params=np.nan, cv_parallel=True, cv_cores=multiprocessing.cpu_count() - 2, *args, **kwargs):
    col_idx_target = list(ds.columns).index(target)

    # Apply filtering in accordance to the enabled feature
    if "enabled" in param_def.columns:
        enabled_idx = list(np.where(param_def["enabled"] == True)[0]) + [col_idx_target]
        ds = ds.iloc[:, enabled_idx]
        col_idx_target = len(enabled_idx) - 1

    # Some methods require normalization
    if is_normalization_required:
        ds = fn_normalize_min_max(ds, param_def)

    if not np.all(np.isnan(method_params)):
        method_params_expanded = pd.DataFrame(list(product(*method_params)))
        
        if len(method_params_expanded) > 1:
            # Create cross-validation folds
            cv_datasets = fn_do_folding(ds)
            
            method_params_performance = []
            
            for conf_idx in range(len(method_params_expanded)):
                method_conf = dict(zip(method_params_expanded.columns, method_params_expanded.iloc[conf_idx]))
                
                if cv_parallel:
                    with multiprocessing.Pool(processes=cv_cores) as pool:
                        folds_model_performance = pool.starmap(
                            lambda f_ds, m_conf, t, l_fit_fun: l_fit_fun(f_ds["train"], target=t, method_params=m_conf, test_ds=f_ds["test"])["performance"],
                            [(f_ds, method_conf, target, type) for f_ds in cv_datasets]
                        )
                else:
                    folds_model_performance = [
                        type(f_ds["train"], target=target, method_params=method_conf, test_ds=f_ds["test"])["performance"]
                        for f_ds in cv_datasets
                    ]
                
                mean_performance = np.mean(folds_model_performance, axis=0)
                partial_performance = np.array(folds_model_performance)
                method_params_performance.append({"mean_performance": mean_performance, "partial_performance": partial_performance})
            
            # Retrieve performance of models built on different folds, find the best, and retrain to whole dataset
            mean_performance = np.array([x["mean_performance"] for x in method_params_performance])
            best_performer_idx = np.argmin(mean_performance, axis=0)[0]
            method_conf = dict(zip(method_params_expanded.columns, method_params_expanded.iloc[best_performer_idx]))
            
            list_models = [
                {
                    "model": type(ds.iloc[:, list(sub_ds_idx) + [col_idx_target]], target=target, method_params=method_conf, *args, **kwargs)["model"],
                    "sub_ds_idx": sub_ds_idx
                }
                for sub_ds_idx in fn_split_space(ds)
            ]
            
            return list_models
        
        # Not enough params combinations for tuning
        list_models = [
            {
                "model": type(ds.iloc[:, list(sub_ds_idx) + [col_idx_target]], target=target, method_params=method_params, *args, **kwargs)["model"],
                "sub_ds_idx": sub_ds_idx
            }
            for sub_ds_idx in fn_split_space(ds)
        ]
        
        return list_models

    list_models = [
        {
            "model": type(ds.iloc[:, list(sub_ds_idx) + [col_idx_target]], target=target, *args, **kwargs)["model"],
            "sub_ds_idx": sub_ds_idx
        }
        for sub_ds_idx in fn_split_space(ds)
    ]

    return list_models

In [48]:
## Function for splitting the space into set of subspaces for learning/fitting many models

def fn_split_space(ds):
    no_subspaces = 1
    if "SUBSPACES_NUMBER" in shared.env["settings"]:
        no_subspaces = shared.env["settings"]["SUBSPACES_NUMBER"]

    size_subspace = np.ceil((ds.shape[1]-1)/no_subspaces)
    if "SUBSPACES_SIZE" in shared.env["settings"]:
        size_subspace = np.ceil((ds.shape[1]-1)*shared.env["settings"]["SUBSPACES_SIZE"])

    overlap_subspaces = False
    if "SUBSPACES_OVERLAP" in shared.env["settings"]:
        overlap_subspaces = shared.env["settings"]["SUBSPACES_OVERLAP"]
    if (ds.shape[1]-1) < (no_subspaces * size_subspace):
        overlap_subspaces = True

    output = []
    if overlap_subspaces:
        for idx in range(no_subspaces):
            col_idx = np.sort(np.random.choice(ds.shape[1]-1, size_subspace, replace=False))
            output.append(col_idx)
    else:
        col_set = np.arange(ds.shape[1]-1)
        for i in range(no_subspaces):
            gen_vector = np.sort(np.random.choice(col_set, size_subspace, replace=False))
            output.append(gen_vector)
            col_set = np.setdiff1d(col_set, gen_vector)

    return output

In [49]:
## Function to predict using GP model
# Params:
# - surrogate_model: model fitted with fn_fit function
# - ds: dataset (tibble)

def fn_predict_gp(surrogate_model, ds):
    pred = surrogate_model.predict(ds)
    
    return {
        'pred': pred,
        'mu': pred.mean(),
        'sigma': np.sqrt(pred.var())
    }

In [50]:
## Function to predict using GP model (MLEGP Library)
# Params:
# - surrogate_model: model fitted with fn_fit function
# - ds: dataset (tibble)

# def fn_predict_mlegp(surrogate_model, ds):
#     pred = predict.gp(surrogate_model, newdata = ds, se.fit = True)
#     return {
#         "pred": pred,
#         "mu": pred[0],
#         "sigma": pred[1]
#     }

def fn_predict_mlegp(surrogate_model, ds):
    pred = predict.gp(surrogate_model, newdata=ds, se_fit=True)
    return {
        "pred": pred,
        "mu": pred[0],
        "sigma": pred[1]
    }

In [51]:
## Function to predict using Random Forest model
# Params:
# - surrogate_model: model fitted with fn_fit function
# - ds: dataset (tibble)


#predict.all=True   ->individual predictions for all trees in the forest

def fn_predict_rf(surrogate_model, ds):
    pred = predict(surrogate_model, newdata=ds, predict_all=True)
    return {
        "pred": pred["individual"],
        "mu": pred["aggregate"],
        "sigma": np.apply_along_axis(np.std, axis=1, arr=pred["individual"]),
        "inbag": surrogate_model["inbag"]
    }

# def fn_predict_rf(surrogate_model, ds):
#     pred = surrogate_model.predict(ds, predict_all=True)
    
#     return {
#         "pred": pred["individual"],
#         "mu": pred["aggregate"],
#         "sigma": np.apply_along_axis(np.std, 1, pred["individual"]),
#         "inbag": surrogate_model.inbag
#     }

In [52]:
## Function to predict using a model
# Params:
# - type: mode type - predict function
# - surrogate_model: model fitted with fn_fit function (list of surrogates)
# - ds: dataset (tibble)

def fn_predict(type, surrogate_model, ds, param_def=None, is_normalization_required=False, **kwargs):
    # Apply filtering in accordance to the enabled feature
    if param_def is not None and "enabled" in param_def.columns:
        enabled_idx = param_def.index[param_def["enabled"]]
        ds = ds.iloc[:, enabled_idx]

    if is_normalization_required and param_def is not None:
        ds = fn_normalize_min_max(ds, param_def)

    # Surrogate model object contains list of lists [surrogate, indices of subspace columns]
    list_predictions = [type(l_model_subspace[0], ds.iloc[:, l_model_subspace[1]], **kwargs) for l_model_subspace in surrogate_model]
    output_headers = list_predictions[0].keys()
    output = {h: [pred[h] for pred in list_predictions] for h in output_headers}

    return output


In [53]:
################################ B OPT##############################

In [54]:
# pip install scikit-optimize
from skopt import gp_minimize, Optimizer
from skopt.space import Real
import numpy as np

def fn_bayes_optimisiation(param_init, param_def, param_addons=[], simulation_fn=None, termination_fn=None, 
                           verbose_output_fn=None, save_object_fn=None, target="inadequacy", surrogate_fit_fn="fn_fit_rf",
                           surrogate_predict_fn="fn_predict_rf", surrogate_utility_fn="fn_utility_ei", 
                           init_sampling_fn="fn_sampling_lhs", sampling_fn="fn_sampling_lhs", 
                           acquisition_fn="fn_optimal_acquisition", selection_fn="fn_pull_optimal_selection", 
                           inner_optimisation_fn="fn_optimisation_quasi_newthon", init_sample_size=10, sample_size=100, 
                           potential_size=1, initialize_only=False, store_iteratively=False, **kwargs):
    
    # Initialize optimizer
    opt = Optimizer(dimensions=[Real(*bounds) for bounds in param_def.values()],
                    base_estimator=surrogate_fit_fn, acq_func=surrogate_utility_fn)
    
    iteration_no = 0
    
    # Generate initial set of params to be simulated
    init_x = np.array([list(param_init.values())])
    init_y = np.array([simulation_fn(**param_init)])
    opt.tell(init_x, init_y)
    
    if verbose_output_fn is not None:
        verbose_output_fn("Initial sample set acquired.", count=init_sample_size)
    
    while not termination_fn(iteration_no, opt):
        # Increase iteration number
        iteration_no += 1
        
        # 1. get the best performer and save its parameters
        x_next = opt.ask()
        y_next = simulation_fn(**dict(zip(param_init.keys(), x_next)))
        opt.tell(x_next, y_next)
        
        if verbose_output_fn is not None:
            verbose_output_fn("Retrieved current most optimal sample.", count=len(x_next))
      
        # Save current state of optimizer
        if store_iteratively and save_object_fn is not None:
            param_space = {"space": np.concatenate([opt.Xi, np.array(opt.yi).reshape(-1, 1)], axis=1)}
            save_object_fn(param_space, "param_space", stage=f"iter_{iteration_no}")
      
    # Return final results
    param_space = {"space": np.concatenate([opt.Xi, np.array(opt.yi).reshape(-1, 1)], axis=1)}
    return param_space

In [ ]:
# 1st try 

In [57]:
def fn_bayes_optimisiation(param_init, param_def, param_addons=[], simulation_fn=None, termination_fn=None, 
                           verbose_output_fn=None, save_object_fn=None, target="inadequacy", surrogate_fit_fn="fn_fit_rf",
                           surrogate_predict_fn="fn_predict_rf", surrogate_utility_fn="fn_utility_ei", 
                           init_sampling_fn="fn_sampling_lhs", sampling_fn="fn_sampling_lhs", 
                           acquisition_fn="fn_optimal_acquisition", selection_fn="fn_pull_optimal_selection", 
                           inner_optimisation_fn="fn_optimisation_quasi_newthon", init_sample_size=10, sample_size=100, 
                           potential_size=1, initialize_only=False, store_iteratively=False, **kwargs):

############################### -  Part A  - ##########################
# Get global settings object
    if "RETRAIN_AFTER" not in shared.env["settings"]:
        shared.env["settings"]["RETRAIN_AFTER"] = 1
    settings_obj = shared.env["settings"]

    # Compile all functions
    try:
        fn_compilation = ["surrogate_fit_fn", "surrogate_predict_fn", "surrogate_utility_fn", "init_sampling_fn", "sampling_fn", "acquisition_fn", "selection_fn", "termination_fn", "inner_optimisation_fn"]
        for fn in fn_compilation:
            exec("%s = %s" % (fn, eval(fn)))
        
        if not callable(simulation_fn):
            simulation_fn = eval(simulation_fn)
    except Exception as e:
        raise e

    # Check if terminal conditions are set up
    if termination_fn is None:
        raise ValueError("Termination function is required! It must return logical value T (terminate)/F (continue). At input get the iteration number, params and inadequacy through iterations")

    iteration_no = 0

    # Assemble the param space
    param_space = {"space": param_init, "definition": param_def}
    param_space.update(param_addons)


    ###############################_-  Part B  -_#########################
    if nrow(param_space['space']) == 0:
        # Generate initial set of params to be simulated! (preferably 10 instances)
        init_sampling_params = {
            'type': init_sampling_fn,
            'param_space': param_space
        }
        init_sampling_params = fn_resolve_fn_params(init_sampling_fn, sample_size=init_sample_size, omit=['target'], **init_sampling_params)
        init_samples = do.call(fn_acquisition, init_sampling_params)
        param_space['space'] = rbind(param_space['space'], init_samples)
        
        if verbose_output_fn is not None:
            verbose_output_fn("Initial sample set acquired.", count=init_sample_size)

        # Simulate initial set of params
        simulation_fn_params = {
            'param_def': param_space['definition']
        }
        simulation_fn_params = fn_resolve_fn_params(simulation_fn, **simulation_fn_params)
        simulation_outcome = apply(param_space['space'], 1, simulation_fn, **simulation_fn_params)
        simulation_outcome = t(matrix(simulation_outcome, ncol=1))
        simulation_outcome.columns = target
        #colnames(simulation_outcome) = target
        param_space['space'] = cbind(param_space['space'], simulation_outcome)
        
        if verbose_output_fn is not None:
            verbose_output_fn("Initial sample set simulated.")
        
        if save_object_fn is not None:
            save_object_fn(param_space, "param_space", stage="initial")
        
    if initialize_only:
        return None

    termination_fn_params = fn_resolve_fn_params(termination_fn, **{'space': param_space})

    ###############################_-  Part C  -_#########################
    # Perform optimization
    while not termination_fn(iteration_no, param_space):
        # Increase iteration number
        iteration_no += 1
        iteration_space = {}

        try:
            # 1. get the best performer and save its parameters
            
            #iteration_space['optimal'] = fn_pull_optimal_samples(space=param_space['space'], selection_fn=selection_fn, target=target, **fn_resolve_fn_params(selection_fn, _omit=["space", "selection_fn", "target"], **locals()))
            #iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, ds=param_space['space'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], ...), target=target, param_def=param_space['definition'], **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], ...))
            #iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, ds=param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], ...), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], ...))
            #iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, param_space['space'], target, param_def=param_space['definition'], *fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], **{}), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], **{}))
            #iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, param_space['space'], target, param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], ...), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], ...))
            #iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], **locals()), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], **locals()))
            iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, ds=param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def"]), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"]))


            #iteration_space['optimal'] = fn_pull_optimal_samples(param_space['space'], selection_fn, target, **fn_resolve_fn_params(selection_fn, omit=["space", "selection_fn", "target"], ...))
            if verbose_output_fn is not None:
                verbose_output_fn("Retrieved current most optimal sample.", count = nrow(iteration_space['optimal']['space']))
        
            # 2. fit surrogate model
            if (iteration_no == 1) or (iteration_no % settings_obj['RETRAIN_AFTER'] == 0):
                iteration_space['surrogate'] = {}
                iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, ds=param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], ...), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], ...))
                param_space['surrogate'] = iteration_space['surrogate']
                if verbose_output_fn is not None:
                    verbose_output_fn("Fitted surrogate model(s).", training_set_size=nrow(param_space['space']))
            else:
                iteration_space['surrogate'] = param_space['surrogate']
                if verbose_output_fn is not None:
                    verbose_output_fn("Loaded previously trained surrogate model(s).", training_set_size=nrow(param_space['space']))

        except Exception as e:
            # handle errors during optimization
            if error_handling_fn is not None:
                error_handling_fn(e, iteration_no, param_space)
            else:
                raise e
            
    ###############################_-  Part D  -_#########################
    # 3. optimise acquisition utility
    iteration_space["potential"] = fn_acquisition(type=acquisition_fn, param_space=param_space, **fn_resolve_fn_params(acquisition_fn, {"type", "param_space"}, iter_space=iteration_space, sample_size=sample_size, omit=[target], sampling_fn=sampling_fn, surrogate_predict_fn=surrogate_predict_fn, surrogate_utility_fn=surrogate_utility_fn, inner_optimisation_fn=inner_optimisation_fn, ...))

    if verbose_output_fn is not None:
        verbose_output_fn("New optimal sample set acquired.")

    # 4. simulate results of the picked-up instances
    simulation_outcome = apply(iteration_space["potential"]["space"], 1, simulation_fn, **fn_resolve_fn_params(simulation_fn, {1}, param_def=param_space["definition"], ...))
    iteration_space["potential"]["space"] = np.concatenate((iteration_space["potential"]["space"], simulation_outcome), axis=1)
    iteration_space["potential"]["space"][:, -1] = target
    if verbose_output_fn is not None:
        verbose_output_fn("Simulation of the most promising samples is completed.")

    if "complementary_utility_fn" in iteration_space["potential"] and len(iteration_space["potential"]["complementary_utility_fn"]) > 0:
        # 4b. optionally simulate results of the picked-up instances by complementary acquisition functions
        compl_space = pd.concat([pd.DataFrame({"select_idx": None, "acq": fn_header, "utility": None, "simulation": None, "iteration": iteration_no}, index=[0], columns=iteration_space["potential"]["pop"].columns.tolist() + ["select_idx", "acq", "utility", "simulation", "iteration"]) for fn_header in iteration_space["potential"]["complementary_utility_fn"]])
        for i, row in iteration_space["potential"]["pop"].iterrows():
            best_val_idx = row[iteration_space["potential"]["complementary_utility_fn"]].argmax()
            eval_set = pd.DataFrame(row[iteration_space["potential"]["pop"].columns.tolist()]).T
            simulation_outcome = apply(eval_set, 1, simulation_fn, **fn_resolve_fn_params(simulation_fn, {1}, param_def=param_space["definition"], ...))
            eval_set["select_idx"] = best_val_idx
            eval_set["acq"] = fn_header
            eval_set["utility"] = row[fn_header]
            eval_set["simulation"] = simulation_outcome
            eval_set["iteration"] = iteration_no
            compl_space.iloc[i] = eval_set.iloc[0]
        
        if "complementary_space" in param_space:
            param_space["complementary_space"] = pd.concat([param_space["complementary_space"], compl_space])
        else:
            param_space["complementary_space"] = compl_space
            
        if verbose_output_fn is not None:
            verbose_output_fn("Simulation of the most promising samples from complementary acquisition functions is completed.") 



    ###############################_-  Part E  -_#########################
    try:
                iteration_space = simulate_fn(param_space)
                param_space.space = pd.concat([param_space.space, pd.DataFrame(iteration_space.potential.space)])
                if verbose_output_fn is not None:
                    verbose_output_fn("Simulated results are added to the global set.")
                    
                if save_object_fn is not None and store_iteratively:
                    save_object_fn(iteration_space, "param_space", stage="optimisation", iteration=iteration_no)
                if save_object_fn is not None:
                    save_object_fn(param_space, "output_object", stage="partial")
                if 'shared.env' in globals() and 'param_space' in globals()['shared.env']:
                    globals()['shared.env']['param_space'] = param_space
            
    except Exception as err:
        if save_object_fn is not None:
            save_object_fn(iteration_space, "param_space", stage="failed_optimisation_iter", iteration=iteration_no)
        if verbose_output_fn is not None:
            verbose_output_fn(f"ERROR: Iteration failed! iteration={iteration_no}, ex={err}")
        raise err
        
    return param_space


SyntaxError: positional argument follows keyword argument (2695344018.py, line 99)

In [ ]:
iteration_space['surrogate']['model'] = fn_fit(surrogate_fit_fn, ds=param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], ...), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], ...))

In [19]:
############
##################
#######################
def fn_bayes_optimisiation(
        param_init,
        param_def,
        param_addons=None,
        simulation_fn=None,
        termination_fn=None,
        verbose_output_fn=None,
        save_object_fn=None,
        target="inadequacy",
        surrogate_fit_fn="fn_fit_rf",
        surrogate_predict_fn="fn_predict_rf",
        surrogate_utility_fn="fn_utility_ei",
        init_sampling_fn="fn_sampling_lhs",
        sampling_fn="fn_sampling_lhs",
        acquisition_fn="fn_optimal_acquisition",
        selection_fn="fn_pull_optimal_selection",
        inner_optimisation_fn="fn_optimisation_quasi_newthon",
        init_sample_size=10,
        sample_size=100,
        potential_size=1,
        initialize_only=False,
        store_iteratively=False,
        **kwargs):
    # Get global settings object
    if "RETRAIN_AFTER" not in shared.env["settings"]:
        shared.env["settings"]["RETRAIN_AFTER"] = 1
    settings_obj = shared.env["settings"]

    # Compile all functions
    for fn_compilation in ["surrogate_fit_fn", "surrogate_predict_fn", "surrogate_utility_fn",
                           "init_sampling_fn", "sampling_fn", "acquisition_fn", "selection_fn",
                           "termination_fn", "inner_optimisation_fn"]:
        try:
            locals()[fn_compilation] = globals()[eval(fn_compilation)]
        except KeyError:
            raise ValueError(f"Function {fn_compilation} not found.")

    if not callable(simulation_fn):
        simulation_fn = globals()[simulation_fn]

    # Check if terminal conditions are set up
    if termination_fn is None:
        raise ValueError("Termination function is required! It must return logical value T (terminate)/F (continiue). "
                         "At input get the iteration number, params and inadequacy through iterations")

    iteration_no = 0

    # Assemble the param space
    param_space = {"space": param_init, "definition": param_def}
    if param_addons is not None:
        param_space.update(param_addons)

    if len(param_space["space"]) == 0:
        # Generate initial set of params to be simulated! (preferable 10 instances)
        init_sample = acquisition_fn(type=init_sampling_fn, param_space=param_space,
                                      **fn_resolve_fn_params(init_sampling_fn, omit=["type", "param_space", target], 
                                                             sample_size=init_sample_size, **kwargs))
        param_space["space"] = np.vstack([param_space["space"], init_sample])
        if verbose_output_fn is not None:
            verbose_output_fn("Initial sample set acquired.", count=init_sample_size)

        # Simulate initial set of params
        simulation_outcome = np.apply_along_axis(simulation_fn, 1, param_space["space"],
                                                 **fn_resolve_fn_params(simulation_fn, omit=[0], 
                                                                        param_def=param_space["definition"], **kwargs))
        param_space["space"] = np.column_stack([param_space["space"], simulation_outcome])
        param_space["target"] = target
        if verbose_output_fn is not None:
            verbose_output_fn("Initial sample set simulated.")
        if save_object_fn is not None:
            save_object_fn(param_space, "param_space", stage="initial")

    if initialize_only:
        return None

    termination_args = fn_resolve_fn_params(termination_fn, omit=["iteration", "space"], **kwargs)

    ##############################Part 2#################################
    
    while not termination_fn(iteration=iteration_no, space=param_space, **termination_args):
    # Increase iteration number
    iteration_no += 1
    iteration_space = {}
    
    try:
        # 1. get the best performer and save its parameters
        iteration_space['optimal'] = fn_pull_optimal_samples(space=param_space['space'], selection_fn=selection_fn, target=target, **fn_resolve_fn_params(selection_fn, omit=['space', 'selection_fn', 'target'], **...))
        if verbose_output_fn is not None:
            verbose_output_fn("Retrieved current most optimal sample.", count=len(iteration_space['optimal']['space']))
      
        # 2. fit surrogate model
        if (iteration_no == 1) or (iteration_no % settings_obj['RETRAIN_AFTER'] == 0):
            iteration_space['surrogate'] = {}
            iteration_space['surrogate']['model'] = fn_fit(type=surrogate_fit_fn, ds=param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=['type', 'ds', 'target', 'param_def', '...'], **fn_resolve_fn_params(surrogate_fit_fn, omit=['type', 'ds', 'target', 'param_def', 'method_params'], **...))
            param_space['surrogate'] = iteration_space['surrogate']
            if verbose_output_fn is not None:
                verbose_output_fn("Fitted surrogate model(s).", training_set_size=len(param_space['space']))
        else:
            iteration_space['surrogate'] = param_space['surrogate']
            if verbose_output_fn is not None:
                verbose_output_fn("Loaded previously trained surrogate model(s).", training_set_size=len(param_space['space']))
      
        # 3. optimise acquisition utility
        iteration_space['potential'] = fn_acquisition(type=acquisition_fn, param_space=param_space, **fn_resolve_fn_params(acquisition_fn, omit=['type', 'param_space'], iter_space=iteration_space, sample_size=sample_size, omit=[target], sampling_fn=sampling_fn, surrogate_predict_fn=surrogate_predict_fn, surrogate_utility_fn=surrogate_utility_fn, inner_optimisation_fn=inner_optimisation_fn, **...), **fn_resolve_fn_params(inner_optimisation_fn, omit=['sampling_fn', 'surrogate_predict_fn', 'surrogate_model', 'surrogate_utility_fn', 'iteration_space', 'param_range', 'param_space', 'sample_size', 'omit'], target=target, **...))
        if verbose_output_fn is not None:
            verbose_output_fn("New optimal sample set acquired.")

IndentationError: expected an indented block (4188850852.py, line 82)

In [20]:




### 2nd try !!!!!
# Perform optimization
while not termination_fn(iteration=iteration_no, space=param_space, **termination_args):
    # Increase iteration number
    iteration_no += 1
    iteration_space = dict()
    
    try:
        # 1. Get the best performer and save its parameters
        iteration_space['optimal'] = fn_pull_optimal_samples(space=param_space['space'], selection_fn=selection_fn, target=target, **fn_resolve_fn_params(selection_fn, omit=["space", "selection_fn", "target"], **params))

        if verbose_output_fn is not None:
            verbose_output_fn("Retrieved current most optimal sample.", count=len(iteration_space['optimal']['space']))
        
        # 2. Fit surrogate model
        if (iteration_no == 1) or (iteration_no % settings_obj['RETRAIN_AFTER'] == 0):
            iteration_space['surrogate'] = dict()
            iteration_space['surrogate']['model'] = fn_fit(type=surrogate_fit_fn, ds=param_space['space'], target=target, param_def=param_space['definition'], **fn_resolve_fn_params(fn_fit, omit=["type", "ds", "target", "param_def", "..."], **params), **fn_resolve_fn_params(surrogate_fit_fn, omit=["type", "ds", "target", "param_def", "method_params"], **params))
            
            param_space['surrogate'] = iteration_space['surrogate']
            
            if verbose_output_fn is not None:
                verbose_output_fn("Fitted surrogate model(s).", training_set_size=len(param_space['space']))
        else:
            iteration_space['surrogate'] = param_space['surrogate']
            
            if verbose_output_fn is not None:
                verbose_output_fn("Loaded previously trained surrogate model(s).", training_set_size=len(param_space['space']))

        # 3. Optimise acquisition utility
        iteration_space['potential'] = fn_acquisition(type=acquisition_fn, param_space=param_space, **fn_resolve_fn_params(acquisition_fn, omit=["type", "param_space"], iter_space=iteration_space, sample_size=sample_size, omit=[target], sampling_fn=sampling_fn, surrogate_predict_fn=surrogate_predict_fn, surrogate_utility_fn=surrogate_utility_fn, inner_optimisation_fn=inner_optimisation_fn, **params), **fn_resolve_fn_params(inner_optimisation_fn, omit=["sampling_fn", "surrogate_predict_fn", "surrogate_model", "surrogate_utility_fn", "iteration_space", "param_range", "param_space", "sample_size", "omit"], target=target, **params))

        if verbose_output_fn is not None:
            verbose_output_fn("New optimal sample set acquired.")
            
            
        # 4. Simulate results of the picked-up instances
        simulation_outcome = simulation_fn(**fn_resolve_fn_params(simulation_fn, iteration_space["potential"]["space"], param_def=param_space["definition"], omit=1))
        iteration_space["potential"]["space"][target] = simulation_outcome
        if verbose_output_fn is not None:
            verbose_output_fn("Simulation of the most promising samples is completed.")

        if "complementary_utility_fn" in iteration_space["potential"] and len(iteration_space["potential"]["complementary_utility_fn"]) > 0:
            # 4b. Optionally simulate results of the picked-up instances by complementary acquisition functions
            compl_space = []

            for fn_header in iteration_space["potential"]["complementary_utility_fn"]:
                best_val_idx = iteration_space["potential"]["pop"][fn_header].idxmax()
                eval_set = iteration_space["potential"]["pop"].loc[best_val_idx, iteration_space["potential"]["space"].columns].to_frame().T
                simulation_outcome = simulation_fn(**fn_resolve_fn_params(simulation_fn, eval_set, param_def=param_space["definition"], omit=1))

                eval_set["select_idx"] = best_val_idx
                eval_set["acq"] = fn_header
                eval_set["utility"] = iteration_space["potential"]["pop"].loc[best_val_idx, fn_header]
                eval_set["simulation"] = simulation_outcome
                eval_set["iteration"] = iteration_no
                compl_space.append(eval_set)

            compl_space = pd.concat(compl_space, ignore_index=True)

            if "complementary_space" in param_space:
                param_space["complementary_space"] = pd.concat([param_space["complementary_space"], compl_space], ignore_index=True)
            else:
                param_space["complementary_space"] = compl_space

            if verbose_output_fn is not None:
                verbose_output_fn("Simulation of the most promising samples from complementary acquisition functions is completed.")

        # 5. Add simulated results to the base set
        param_space["space"] = pd.concat([param_space["space"], iteration_space["potential"]["space"]], ignore_index=True)
        if verbose_output_fn is not None:
            verbose_output_fn("Simulated results are added to the global set.")

        # Store required objects
        if save_object_fn is not None and store_iteratively:
            save_object_fn(iteration_space, "param_space", stage="optimisation", iteration=iteration_no)
        if save_object_fn is not None:
            save_object_fn(param_space, "output_object", stage="partial")

        # (Optional) Assuming 'shared' is a dictionary that represents the shared environment
        if "shared" in globals() and "param_space" in shared:
            shared["param_space"] = param_space
            


SyntaxError: unexpected EOF while parsing (3786701723.py, line 82)

In [ ]:

############### Corrected ####################### 

# 4. Simulate results of the picked-up instances
simulation_outcome = apply(iteration_space['potential']['space'], axis=1, func=simulation_fn, **fn_resolve_fn_params(simulation_fn, omit=1, param_def=param_space['definition'], **kwargs))
iteration_space['potential']['space'][target] = simulation_outcome
if verbose_output_fn is not None:
    verbose_output_fn("Simulation of the most promising samples is completed.")

if 'complementary_utility_fn' in iteration_space['potential'] and len(iteration_space['potential']['complementary_utility_fn']) > 0:
    # 4b. Optionally simulate results of the picked-up instances by complementary acquisition functions
    compl_space = pd.concat([
        pd.DataFrame([{
            "select_idx": np.argmax(iteration_space['potential']['pop'][fn_header]),
            "acq": fn_header,
            "utility": iteration_space['potential']['pop'].loc[np.argmax(iteration_space['potential']['pop'][fn_header]), fn_header],
            "simulation": apply(eval_set, axis=1, func=simulation_fn, **fn_resolve_fn_params(simulation_fn, omit=1, param_def=param_space['definition'], **kwargs)),
            "iteration": iteration_no
        }])
        for fn_header in iteration_space['potential']['complementary_utility_fn']
    ], ignore_index=True)

    if 'complementary_space' in param_space:
        param_space['complementary_space'] = pd.concat([param_space['complementary_space'], compl_space], ignore_index=True)
    else:
        param_space['complementary_space'] = compl_space

    if verbose_output_fn is not None:
        verbose_output_fn("Simulation of the most promising samples from complementary acquisition functions is completed.")

# 5. Add simulated results to the base set
param_space['space'] = pd.concat([param_space['space'], iteration_space['potential']['space']], ignore_index=True)
if verbose_output_fn is not None:
    verbose_output_fn("Simulated results are added to the global set.")

# Store required objects
if save_object_fn is not None and store_iteratively:
    save_object_fn(iteration_space, "param_space", stage="optimisation", iteration=iteration_no)
if save_object_fn is not None:
    save_object_fn(param_space, "output_object", stage="partial")

shared["param_space"] = param_space

In [58]:
### its not the best
## i can find some interesting lines here


import random
import numpy as np
from functools import partial
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize

def fn_bayes_optimisiation(param_init, param_def, param_addons=None, simulation_fn=None, termination_fn=None, verbose_output_fn=None,
                           save_object_fn=None, target="inadequacy", surrogate_fit_fn=None, surrogate_predict_fn=None, 
                           surrogate_utility_fn=None, init_sampling_fn=None, sampling_fn=None, acquisition_fn=None, 
                           selection_fn=None, inner_optimisation_fn=None, init_sample_size=10, sample_size=100, 
                           potential_size=1, initialize_only=False, store_iteratively=False, **kwargs):
    
    # Get global settings object
    if "RETRAIN_AFTER" not in shared.env["settings"]:
        shared.env["settings"]["RETRAIN_AFTER"] = 1
    settings_obj = shared.env["settings"]
    
    # Compile all functions
    fn_compilation = ["surrogate_fit_fn", "surrogate_predict_fn", "surrogate_utility_fn", "init_sampling_fn", "sampling_fn", 
                      "acquisition_fn", "selection_fn", "termination_fn", "inner_optimisation_fn"]
    for fn_name in fn_compilation:
        if eval(fn_name) is None:
            raise ValueError(f"{fn_name} is required.")
        else:
            exec(f"{fn_name} = {eval(fn_name)}")
    
    if simulation_fn is None or not callable(simulation_fn):
        raise ValueError("simulation_fn is required and must be a callable function.")
    
    # Check if terminal conditions are set up
    if termination_fn is None:
        raise ValueError("Termination function is required!")
    
    iteration_no = 0
    
    # Assemble the parameter space
    param_space = {"space": np.array(param_init), "definition": param_def}
    if param_addons is not None:
        param_space.update(param_addons)
    
    if param_space["space"].shape[0] == 0:
        # Generate initial set of parameters to be simulated
        init_params = do_call(acquisition_fn, type=init_sampling_fn, param_space=param_space, 
                              **fn_resolve_fn_params(init_sampling_fn, omit=["type", "param_space", "target"], sample_size=init_sample_size, **kwargs))
        param_space["space"] = np.vstack([param_space["space"], init_params])
        if verbose_output_fn is not None:
            verbose_output_fn("Initial sample set acquired.", count=init_sample_size)
        
        # Simulate initial set of parameters
        simulation_outcome = np.apply_along_axis(lambda params: simulation_fn(params, **fn_resolve_fn_params(simulation_fn, 
                                                                    omit=[0], param_def=param_space["definition"], **kwargs)), 
                                                 1, param_space["space"])
        param_space["space"] = np.hstack([param_space["space"], simulation_outcome.reshape(-1, 1)])
        param_space["target"] = target
        if verbose_output_fn is not None:
            verbose_output_fn("Initial sample set simulated.")
        if save_object_fn is not None:
            save_object_fn(param_space, "param_space", stage="initial")
    
    if initialize_only:
        return None
    
    # Check termination arguments
    if termination_fn is not None:
        termination_args = fn_resolve_fn_params(termination_fn, omit=["iteration", "space"], **kwargs)
    else:
        raise ValueError("Termination function is required!")
    

In [ ]:
################################ B OPT##############################

In [67]:
## Function: assemble prediction traits that helps further applying acquisition and utility functions
## Params:
## - prediction_obj: object containning predictions return from surrogate_model_fn

def fn_assemble_prediction_traits(prediction_obj):
    prediction_traits = {}
    
    mu = np.apply_along_axis(np.mean, 1, np.hstack([np.matrix(x).reshape(-1,1) for x in prediction_obj['mu']]), nan_policy='omit')
    prediction_traits['mu'] = mu
    
    sigma = np.apply_along_axis(np.mean, 1, np.hstack([np.matrix(x).reshape(-1,1) for x in prediction_obj['sigma']]), nan_policy='omit')
    prediction_traits['sigma'] = sigma
    
    prediction_traits['values'] = np.hstack(prediction_obj['pred'])
    
    custom = {k: v for k, v in prediction_obj.items() if k not in ['mu', 'sigma', 'pred']}
    prediction_traits['custom'] = custom
    
    return prediction_traits

In [68]:
def fn_assemble_prediction_traits_single(prediction_obj):
    prediction_traits = {}
    prediction_traits['mu'] = prediction_obj['mu']
    prediction_traits['sigma'] = prediction_obj['sigma']
    prediction_traits['values'] = prediction_obj['pred']
    
    prediction_traits['custom'] = {k:v for k,v in prediction_obj.items() if k not in ["mu","sigma","pred"]}
    
    return prediction_traits

In [70]:
## Function: perform selection of optimal samples
## Params:
## - space: param space to be selected from
## - target: variable name to be arranged with
## - objective_fn: function that will take a vector and return subset of that vector that optimises certain objective. Default it minimum
## - selection_fn: function that will perform selection after optimal have been selected. This primarily refers to selector that will 
##                 selected either all or single sample, with or without randomization

def fn_pull_optimal_samples(space, target, objective_fn=min, selection_fn=None, **kwargs):
    best_performer = {}
    best_performer['idx'] = [i for i in range(len(space)) if space[i][target] == objective_fn([s[target] for s in space])]
    best_performer['space'] = [space[i] for i in best_performer['idx']]
    
    if selection_fn is not None:
        selected_space = selection_fn(best_performer['space'], **kwargs)
        best_performer['space'] = selected_space['space']
        best_performer['idx'] = [best_performer['idx'][i] for i in selected_space['idx']]
        
    best_performer['value'] = objective_fn([s[target] for s in best_performer['space']])
    
    return best_performer

In [71]:
## Function: perform selection of one or many samples form optimally pulled
## Params:
## - space: optimal space selected
## - random: flag indicating randomisation before selection
## - selection_size: number of element to return from the beginning

def fn_pull_optimal_selection(space, randomized_selection=False, selection_size=0):
    space_nrow = space.shape[0]
    idx = np.arange(space_nrow)
    if randomized_selection:
        idx = np.random.permutation(space_nrow)
        space = space[idx, :]
    
    if selection_size <= 0 or selection_size >= space_nrow:
        return {'space': space, 'idx': idx}
    
    return {'space': space[:selection_size, :], 'idx': idx[:selection_size]}

In [72]:
## Function: helper function that resolve parameters of a function into a list of values, based on current environment

def fn_resolve_fn_params(fn, *args, **kwargs):
    # Get arguments list of the provided function
    args_list = list(inspect.signature(fn).parameters.keys())

    # Get all parameters provided in this function call
    params_list = list(args) + list(kwargs.values())
    params_names = list(kwargs.keys())

    # Omit args that are requested not to be set
    if isinstance(args[0], int):
        omit = args[0]
        args_list = [arg for arg in args_list if arg not in omit]
    else:
        omit = args[0]
        args_list = [arg for arg in args_list if arg not in omit]

    # If no other args needed, return empty
    if not args_list:
        return args_list

    # Resolve parameter values
    for i, arg in enumerate(args_list):
        if arg in params_names:
            arg_value = params_list[params_names.index(arg)]
            args_list[i] = arg_value

    return args_list

In [73]:
def fn_termination_max_iterations(iteration, space, max_iterations):
    return iteration > max_iterations